# k近傍法およびLoF(Local Outlier Factor) による時系列異常検知
サンプル密度を用いて外れ検知を行う方法は，次元がそれほど多くない場合には直観的にわかりやすい方法です。k近傍法(KNN)とそれを一般化したLoFがよく使われています。

### 必要なモジュールのimport

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.neighbors import NearestNeighbors #k近傍法
from sklearn.neighbors import LocalOutlierFactor #LoF
from scipy import fftpack
from scipy import signal

### セグメントの切り出し関数

In [ ]:
def getcep(lst):
    win= signal.hann(len(lst))  
    spec_db = np.log10(fftpack.fft(lst*win))                          # 時間波形をフーリエ変換してスペクトルにする
    ceps_db = np.real(fftpack.ifft(spec_db))   
    #index=int(len(lst)/10) #低次の 1/10 までに限定する
    #ceps_db[index:len(ceps_db)] = 0 
    ceps_db[0]=0 #直流分を除去
    return spec_db, ceps_db

In [ ]:
def segdata(lst, dim): #lst:１次元系列　dim:セグメントの幅
    segs = np.empty((0,dim), float)#0×dimの空配列
    ceps = np.empty((0,dim), float)#0×dimの空配列
    for i in range(lst.size - dim + 1):#1つづつずらしながらセグメントをつくっている。最後のセグメントの開始点は lst.size-dim
        spec, cep= getcep(lst[i:i+dim])
        
        seg=np.array(lst[i:i+dim][::-1].reshape((1,-1))) #セグメントの切り出し，時系列反転，appendのための2次ベクトル化
        cep=cep.reshape((1,-1))
        ceps = np.append(ceps,cep,axis=0)
        segs = np.append(segs,seg,axis=0)

    return segs, ceps

### データ読み込み，パラメータ設定
 Keoghらの心電図のデータ  http://www.cs.ucr.edu/~eamonn/discords/qtdbsel102.txt
 Keogh, E., Lin, J. and Fu, A.: HOT SAX : Efficiently Finding the Most Unusual Time Series Subsequence, in Proceedings of the Fifth IEEE International Conference on Data Mining, ICDM 05, pp.226-233.

In [ ]:
def getdata():
  !wget "www.dropbox.com/s/x3fmb9mxr4xkip3/qtdbsel102.txt" #ローカルにコピーしてくる
  LEN=3000  #分析区間

  SP=0         #学習用データの開始点
  AP=3000   #テスト用データの開始点　個のデータの場合 4250ポイント付近に異常がある
  data = np.loadtxt("qtdbsel102.txt",delimiter="\t")
  print("データ数:",data.shape[0],"  次元数:",data.shape[1])

  #元データは3次元の時系列，3次のデータ(indexとしては2)を指定して学習/テストデータに分割
  train_org = data[SP:SP+LEN, 2]      #学習用データとして 1～2999サンプル区間を使用
  test_org  = data[AP:AP+LEN, 2]    #テスト用データとして3000～5999サンプルを使用
  
  #x軸
  x=np.arange(SP,SP+LEN)

  return x, train_org, test_org

In [ ]:
x, train_org, test_org = getdata()

plt.plot(x,train_org)
plt.title("train")
plt.legend()
plt.show()
plt.plot(x,test_org)
plt.title("test with anomaly")
plt.legend()
plt.show()

In [ ]:
WLEN=256#セグメントのサイズ
train_seg, train_cep= segdata(train_org, WLEN)
test_seg, test_cep = segdata(test_org, WLEN)

In [ ]:
plt.plot(train_cep[0])

## k近傍学習（学習データ）

In [ ]:
nk = 1#近傍数

#波形の場合
knn = NearestNeighbors(n_neighbors=nk)
knn.fit(train_seg)

#ケプストラムの場合
knn_cep=NearestNeighbors(n_neighbors=nk)
knn_cep.fit(train_cep)

## LoF

In [ ]:
lof = LocalOutlierFactor(n_neighbors=nk,
                           novelty=True,
                           contamination=0.1)

lof_cep = LocalOutlierFactor(n_neighbors=nk,
                           novelty=True,
                           contamination=0.1)

lof.fit(train_seg) # train_dataは正常データが大多数であるような訓練データ
lof_cep.fit(train_cep)

## 未知データに対する評価

## knnの結果

In [ ]:
# knn.fitで使った学習データを用いてk近傍距離(0次に格納される)を求める
dist = knn.kneighbors(test_seg)[0]
dist_cep= knn_cep.kneighbors(test_cep)[0]

# 最大距離で正規化しておく。つまり0～1.0にboundする。
dist= dist/np.max(dist)
dist_cep = dist_cep/np.max(dist_cep)

## LoFの結果

In [ ]:
# lof
prediction = lof.predict(test_seg) # テストデータに対する予測
prediction_cep = lof_cep.predict(test_cep)

score = lof.score_samples(test_seg) # テストデータの異常度
score_cep = lof_cep.score_samples(test_cep)

score=score/np.min(score)
score_cep=score_cep/np.min(score_cep)

## 波形を直接用いた異常検知の結果

In [ ]:
#テストデータ
plt.title("KNN result")
outlier_rows = [i for i in range(len(dist)) if dist[i]>0.4]
for c in outlier_rows:
    plt.axvspan(c, c, color = "skyblue")

plt.plot(test_org)
plt.show()

plt.title("LoF result")
outlier_rows = [i for i in range(len(score)) if score[i]>0.4]
for c in outlier_rows:
    plt.axvspan(c, c, color = "coral")
plt.plot(test_org)
plt.show()


#距離データ
plt.title("distance")
plt.plot(dist,label="knn")
plt.plot(score,label="lof")
plt.legend()
plt.show()

## ケプストラムを用いた異常検知の結果

In [ ]:
#テストデータ
plt.title("KNN cep result")
outlier_rows = [i for i in range(len(dist_cep)) if dist_cep[i]>0.9]
for c in outlier_rows:
    plt.axvspan(c, c, color = "skyblue")

plt.plot(test_org)
plt.show()

plt.title("LoF cep result")
outlier_rows = [i for i in range(len(score_cep)) if score_cep[i]>0.4]
for c in outlier_rows:
    plt.axvspan(c, c, color = "coral")
plt.plot(test_org)
plt.show()


#距離データ
plt.title("distance")
plt.plot(dist_cep,label="knn")
plt.plot(score_cep,label="lof")
plt.legend()
plt.show()